In [2]:
import numpy as np
import time

In [10]:
# Markov Decision Process with two states and two actions
# T : S x A x S -> [0,1]
# R : S x A x S -> R
T = np.array([[[0.1, 0.9], [0.2, 0.8]], [[0.3, 0.7], [0.4, 0.6]]])
R = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
S = {0, 1}
A = {0, 1}
policy = {0: 1, 1: 0}
gamma = 0.1


In [42]:
def evaluate_policy(policy, S,T, R, gamma, max_iter=1000, tol=1e-3, method = 'iter', print_iters = False):
    T_policy = np.stack([T[s,policy[s]] for s in (S)])
    R_policy = np.array([R[s,policy[s]] for s in (S)])
    R_exp_policy = np.sum(np.multiply(R_policy, T_policy),axis=1)
    print(f'\n{R_exp_policy/(1-gamma)}\n')  
    # if method == 'fp' or max_iter * (len(S)**2) > len(S)**2.4:
    #     return np.linalg.inv(np.eye(len(S)) - gamma * T_policy).dot(R_exp_policy)

    #iterative
    V_policy = np.zeros(len(S))
    iter = 0
    while iter < max_iter:
        V_policy_new = R_exp_policy + gamma * T_policy.dot(V_policy)
        if np.linalg.norm(V_policy_new - V_policy) < tol:
            break
        V_policy = V_policy_new
        iter += 1
    
    if print_iters:
        print('Number of iterations: ', iter)
        
    return V_policy


In [43]:
def policy_iteration(S, A, R, T, gamma, max_iter=1000, tol=1e-3, print_iters = False):
    policy = np.random.randint(0,len(S),len(S), dtype=int)
    iter = 0
    t = time.time()
    V_policy = evaluate_policy(policy, S, T, R, gamma, method = 'fp', print_iters = False)
    while iter < max_iter:
        for s in S:
            policy[s] = np.argmax([ np.sum([T[s,a,s1] * (R[s,a,s1] + gamma*V_policy[s1]) for s1 in S]) for a in A])
        iter += 1
        V_policy_new = evaluate_policy(policy, S, T, R, gamma, method = 'fp', print_iters = False)
        if np.linalg.norm(V_policy_new - V_policy) < tol:
            break
        V_policy = V_policy_new
    if print_iters:
        print(f'Time elapsed: {time.time() - t} seconds')
        print('Number of iterations: ', iter)
        print('Policy Value: ', V_policy)
    return policy

In [44]:
def value_iteration(S, A, R, T, gamma, tol = 1e-4, max_iters = 10000, print_iters = False):
    value = np.zeros(len(S))
    iter = 0
    delta = -np.inf
    t = time.time()
    while True and iter < max_iters:
        val_old = value
        delta = 0
        for s in S:
            val_old = value[s]
            value[s] = np.max([np.sum([T[s,a,s1] * (R[s,a,s1] + gamma*value[s1]) for s1 in S]) for a in A])
            delta = max(delta, np.abs(val_old - value[s]))
        if delta < tol * (1 - gamma) / gamma:
            break
        iter += 1
    if print_iters:
        print(f'Time taken: {time.time() - t} seconds')
        print('Number of iterations: ', iter)
        print('Value: ', value)
        print('Delta: ', delta)
    policy = np.zeros(len(S), dtype=int)
    for s in S:
        policy[s] = np.argmax([np.sum([T[s,a,s1] * (R[s,a,s1] + gamma*value[s1]) for s1 in S]) for a in A])
    
    return policy, value
    

In [45]:
value_iteration(S, A, R, T, gamma, print_iters = True, tol = 1e-2,max_iters = 100000)

Time taken: 0.0003192424774169922 seconds
Number of iterations:  2
Value:  [4.54952051 8.2766788 ]
Delta:  0.05336051200000025


(array([1, 1]), array([4.54952051, 8.2766788 ]))

In [47]:
policy_iteration(S, A, R, T, gamma, max_iter=1000, tol=1e-8, print_iters = True)


[4.22222222 6.33333333]


[4.22222222 8.44444444]


[4.22222222 8.44444444]

Time elapsed: 0.002187967300415039 seconds
Number of iterations:  2
Policy Value:  [4.5526736 8.2781632]


array([1, 1])

In [48]:
t = time.time()
print(evaluate_policy(policy, S, T, R, gamma, max_iter = 10))
print(time.time() - t)
t = time.time()
print(evaluate_policy(policy, S, T, R, gamma))
print(time.time() - t)


[4.22222222 8.44444444]

[4.5526736 8.2781632]
0.0029060840606689453

[4.22222222 8.44444444]

[4.5526736 8.2781632]
0.0008909702301025391


In [41]:
policy = {0: 0, 1: 0}
print(evaluate_policy(policy, S, T, R, gamma, max_iter = 10, print_iters = True))
policy = {0: 0, 1: 1}
print(evaluate_policy(policy, S, T, R, gamma, max_iter = 10))
policy = {0: 1, 1: 0}
print(evaluate_policy(policy, S, T, R, gamma, max_iter = 10))
policy = {0: 1, 1: 1}
print(evaluate_policy(policy, S, T, R, gamma, max_iter = 10))


[2.11111111 6.33333333]

[2.48366013 6.20915033]

[2.11111111 8.44444444]

[2.66450917 8.19848975]

[4.22222222 6.33333333]

[4.38943894 6.27062706]

[4.22222222 8.44444444]

[4.55337691 8.2788671 ]
